In [1]:
from openai import OpenAI
from dotenv import load_dotenv,find_dotenv
import os

In [2]:
_ : bool = load_dotenv(find_dotenv("E:/Python/.env"))
api_key = os.environ.get("OPENAI_API_KEY")

In [3]:
name = os.environ.get("MY_NAME")
print(name)

Asif


In [4]:
client: OpenAI = OpenAI(api_key=api_key)

### Backend API or External API (Hard Coded functions)

In [5]:
import json

In [6]:
def get_current_weather(location:str, unit:str)->str:
    if "Islamabad" in location.lower():
        return json.dumps(
            {
                "location": "Islamabad",
                "temperature": "10",
                "unit": "celsius"
            }
        )
    elif "Karachi" in location.lower():
        return json.dumps(
            {
                "location": "Karachi",
                "temperature": "10",
                "unit": "celsius"
            }
        )
    elif  "Lahore" in location.lower():
        return json.dumps(
            {
                "location": "Lahore",
                "temperature": "20",
                "unit": "celsius"
            }
        )
    else:
        return json.dumps(
            {
                "location": location,
                "temperature": "Unknown"
            }
        )



### Types

https://github.com/openai/openai-python/tree/main/src/openai/types/chat


In [10]:
from openai.types.chat.chat_completion import ChatCompletion,ChatCompletionMessage
import json
from IPython.display import display  # Assuming you are using IPython


def run_conversation(main_request: str)->str:
    #Step 1 send the conversation and available function to the model
    messages = [
        {
            "role": "user",
            "content": main_request,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {   "type": "string",
                                        "description": "The city and state, e.g. San Francisco, CA"
                                    },
                        "unit":     {   "type": "string",
                                        "enum": ["celsius", "fahrenheit"]
                                    },
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    
    # First Request
    response: ChatCompletion = client.chat.completions.create(
        model= "gpt-3.5-turbo-1106",
        messages= messages,
        tools= tools,
        tool_choice= "auto", # auto is default, but we'll be explicit
    )
    response_message: ChatCompletion = response.choices[0].message
    display(" *First Response: ", dict(response_message))
    
    tool_calls = response_message.tool_calls
    display("* First Response Tool Calls: ", list(tool_calls))
    
    #Step 2 Check if the model want to call a function
    if tool_calls:
        #Step 3 Call the function
        #Note: The JSON response may not always be valid, be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
            #Only one function in this you can have multiple function
        }
        
    #Step 4 Send the info for each function call and function response to the model
        messages.append(response_message)
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                    location= function_args.get("location"),
                    unit = function_args.get("unit"),
                    )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                })
            
    display("* Second Request Message: ", list(messages))
            
    print("Messages before second API call:", messages)
            
    second_response: ChatCompletion = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=messages,
            )
            
    print("* Second Response: ", dict(second_response))
    return second_response.choices[0].message.content

In [11]:
run_conversation("What's the weather like in Islamabad, Lahore and Karachi?")

' *First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_GCcWKjeaJZAAmovjrJ6Od1O7', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_x1aLmEkc9NbJNfBvYBbLheL6', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_mFZCmOcQCubJbEeGTIljDuZS', function=Function(arguments='{"location": "Karachi", "unit": "celsius"}', name='get_current_weather'), type='function')]}

'* First Response Tool Calls: '

[ChatCompletionMessageToolCall(id='call_GCcWKjeaJZAAmovjrJ6Od1O7', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_x1aLmEkc9NbJNfBvYBbLheL6', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_mFZCmOcQCubJbEeGTIljDuZS', function=Function(arguments='{"location": "Karachi", "unit": "celsius"}', name='get_current_weather'), type='function')]

'* Second Request Message: '

[{'role': 'user',
  'content': "What's the weather like in Islamabad, Lahore and Karachi?"},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GCcWKjeaJZAAmovjrJ6Od1O7', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_x1aLmEkc9NbJNfBvYBbLheL6', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_mFZCmOcQCubJbEeGTIljDuZS', function=Function(arguments='{"location": "Karachi", "unit": "celsius"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_GCcWKjeaJZAAmovjrJ6Od1O7',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Islamabad", "temperature": "Unknown"}'},
 {'tool_call_id': 'call_x1aLmEkc9NbJNfBvYBbLheL6',
  'role': 'tool',
  'name': 'get_c

Messages before second API call: [{'role': 'user', 'content': "What's the weather like in Islamabad, Lahore and Karachi?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_GCcWKjeaJZAAmovjrJ6Od1O7', function=Function(arguments='{"location": "Islamabad", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_x1aLmEkc9NbJNfBvYBbLheL6', function=Function(arguments='{"location": "Lahore", "unit": "celsius"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_mFZCmOcQCubJbEeGTIljDuZS', function=Function(arguments='{"location": "Karachi", "unit": "celsius"}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_GCcWKjeaJZAAmovjrJ6Od1O7', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "Islamabad", "temperature": "Unknown"}'}, {'tool_call_id': 'call_x1aLmEkc9NbJNfBvYBbLheL6', 'role': 'too

"I'm sorry, but I am unable to retrieve the current weather information for Islamabad, Lahore, and Karachi at the moment. You may want to check a reliable weather website or app for the most up-to-date information."